In [1]:
# import sys
# import os
# import tiktoken
# from ai_models.anthropic_model import AnthropicModel

# tokenizer = tiktoken.get_encoding("cl100k_base")

# # ("claude-3-haiku-20240307", 0.25, 1.25, rpm_limit, 50_000, 5_000_000),
# model = AnthropicModel("claude-3-haiku-20240307", rpm_limit=50, tpm_limit=50_000, tpd_limit=5_000_000, tokenizer=tokenizer, input_cost_per_million=0.25, output_cost_per_million=1.25)

# response = model.predict("What is the capital of Poland?")
# print(response)


In [2]:
import time
import vertexai
import os
from google.oauth2 import service_account
from vertexai.batch_prediction import BatchPredictionJob
from dotenv import load_dotenv
load_dotenv()
project_id = os.getenv("GOOGLE_CLOUD_PROJECT")
location = os.getenv("GOOGLE_CLOUD_LOCATION")
input_bucket = os.getenv("GOOGLE_CLOUD_INPUT_BUCKET")
output_bucket = os.getenv("GOOGLE_CLOUD_OUTPUT_BUCKET")
credentials_path = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")

print(project_id, location, input_bucket, output_bucket, credentials_path)

credentials = service_account.Credentials.from_service_account_file(
    credentials_path,
    scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

# Print service account email
print(f"Service Account: {credentials.service_account_email}")

vertexai.init(
    project=project_id,
    location=location,
    credentials=credentials,
)


input_uri = "gs://cloud-samples-data/batch/prompt_for_batch_gemini_predict.jsonl"
output_uri = f"gs://{output_bucket}/gemini-1.5-flash-002"

# Submit a batch prediction job with Gemini model
batch_prediction_job = BatchPredictionJob.submit(
    source_model="gemini-1.5-flash-002",
    input_dataset=input_uri,
    output_uri_prefix=output_uri,
)

# Check job status
print(f"Job resource name: {batch_prediction_job.resource_name}")
print(f"Model resource name with the job: {batch_prediction_job.model_name}")
print(f"Job state: {batch_prediction_job.state.name}")

# Refresh the job until complete
while not batch_prediction_job.has_ended:
    time.sleep(5)
    batch_prediction_job.refresh()

# Check if the job succeeds
if batch_prediction_job.has_succeeded:
    print("Job succeeded!")
else:
    print(f"Job failed: {batch_prediction_job.error}")

# Check the location of the output
print(f"Job output location: {batch_prediction_job.output_location}")

# Example response:
#  Job output location: gs://your-bucket/gen-ai-batch-prediction/prediction-model-year-month-day-hour:minute:second.12345

alert-maker-440321-u0 us-central1 magtest-bucket magtest-bucket C:\Users\dkwar\Desktop\ks\alert-maker-440321-u0-d0e4a63da338.json
Service Account: vertex-ai-service@alert-maker-440321-u0.iam.gserviceaccount.com
BatchPredictionJob created. Resource name: projects/524644790903/locations/us-central1/batchPredictionJobs/6407167993821790208
To use this BatchPredictionJob in another session:
job = batch_prediction.BatchPredictionJob('projects/524644790903/locations/us-central1/batchPredictionJobs/6407167993821790208')
View Batch Prediction Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/6407167993821790208?project=524644790903
Job resource name: projects/524644790903/locations/us-central1/batchPredictionJobs/6407167993821790208
Model resource name with the job: publishers/google/models/gemini-1.5-flash-002
Job state: JOB_STATE_PENDING
Job succeeded!
Job output location: gs://magtest-bucket/gemini-1.5-flash-002/prediction-model-2024-11-04T19:27:22.129